In [1]:

import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import pickle

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

e:\Diploma\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
# Load encoding model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name).to(device)

In [3]:
# Convert prompts to a Python list (if not already)
i = 1
for chunk_path in ['./datasets/processed_chunk_1.csv', './datasets/processed_chunk_2.csv', './datasets/processed_chunk_3.csv', './datasets/processed_chunk_4.csv', './datasets/processed_chunk_5.csv']:
    chunk = pd.read_csv(chunk_path)
    prompts_list = chunk['prompt'].astype(str).tolist()

    # We will encode in batches to avoid running out of memory on large datasets
    batch_size = 512
    embeddings = []

    for start_idx in range(0, len(prompts_list), batch_size):
        batch_prompts = prompts_list[start_idx:start_idx + batch_size]
        batch_embeddings = model.encode(batch_prompts, 
                                        batch_size=batch_size, 
                                        show_progress_bar=True, 
                                        convert_to_numpy=True, 
                                        device=device)
        embeddings.append(batch_embeddings)
    with open(f'./datasets/processed_chunk_{i}.pickle', 'wb') as chunk_dump:
        pd.to_pickle(embeddings, chunk_dump)
    i+=1

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.53s/it]


In [5]:
chunk = None

In [4]:
embeddings = None

In [ ]:
import pickle
import psycopg2
from psycopg2.extras import execute_batch
conn = psycopg2.connect(
    dbname='VectorizedDataset',
    user='postgres',
    password='fgfaswzxc45d',
    host='localhost',
    port=5432,
    options="-c client_encoding=UTF8"
)
cursor = conn.cursor()


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 61: invalid continuation byte